### **Config**

In [4]:
import os
import sys

sys.path.insert(0, '/home/marco/epfl/magma/')
import config

In [5]:
# General configurations

MODEL = 'pegasus'

RE_SPLITTER = '\.(?!\d)|\n'     # do we split sentences of paragraphs?
                                # use '\.|\n' or '\n', respectively

OUTPUT_PATH = config.DATASET_PATH+'chunk_chapter/'+MODEL+'/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

### **Init**

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import re
import pandas as pd
import gensim

if 'pegasus' in MODEL:
    from transformers import PegasusTokenizer
    tokenizer =\
        PegasusTokenizer.from_pretrained('google/pegasus-large')
elif 'bart' in MODEL:
    from transformers import BartTokenizer
    tokenizer =\
        BartTokenizer.from_pretrained('facebook/bart-large-cnn')
elif 't5' in MODEL:
    from transformers import T5Tokenizer
    tokenizer=\
        T5Tokenizer.from_pretrained('t5-large')

### **Karger Books Base Dataset**

In [8]:
base_dataset = config.MAGMA_DIR+'datasets/karger_books_base/df.csv'
df = pd.read_csv(base_dataset)
df = df.set_index(['book', 'chapter', 'section', 'subsection'])
df.bullets = df.bullets.map(eval, na_action='ignore')

### **Preprocessing**

#### Preprocessing

* Split based on RE_SPLITTER
* Explode the dataset
* Remove unwanted chars at beginning or end of sentence
* Remove multiple spaces
* Remove long words (> config.TOKEN_MAX_LEN chars)
* Remove short sentences / paragraphs (< config.PARA_MIN_LEN tokens)

In [9]:
# Split in sentences / paragraphs based on RE_SPLITTER
df.text =\
    df.text.map(lambda x: [p.strip() for p in re.split(RE_SPLITTER, x) if p!=''],
                na_action='ignore')
    
# explode to get one row for each paragraph /sentence
df = df.explode('text')
df = df.rename(columns={'text': 'para'})
df = df.dropna()

# Remove unwanted chars at beginning or end of sentence
df.para = df.para.map(lambda p: p.lstrip('.,;:-)] \n'))
df.para = df.para.map(lambda p: p.rstrip('.,;:-([ \n'))

# Remove multiple spaces
df.para = df.para.map(lambda p:
    re.sub('\s+', ' ', p).strip())

# Remove long words (> config.TOKEN_MAX_LEN chars)
def para2words(para):
    return gensim.utils.simple_preprocess(
        para, deacc=True, max_len=config.TOKEN_MAX_LEN)
df['para_proc'] = df.para.map(para2words)

# Remove short sentences / paragraphs (< config.PARA_MIN_LEN tokens)
df.loc[df.para_proc.map(len) <\
    config.PARA_MIN_LEN, 'para_proc'] = np.nan

df = df.dropna()

### **Chunk Chapters**

#### Chunk text

In [10]:
df = df.groupby(level=[0, 1], sort=False).agg(
    {'para': lambda t: [p+' . ' for p in t],
     'bullets': lambda b: list(b)[0]})
df = df.rename(columns={'para': 'text'})

In [11]:
def chunk_text(text_sent):
    # calculate number of tokens per sentence and total
    num_tok_sent = [len(tokenizer.tokenize(sent)) for sent in text_sent]
    num_tok = np.sum(num_tok_sent)

    # calculate chunk dimension to fit into model
    n = int(np.ceil(num_tok / config.MODEL_MAX_LEN))
    len_chunk = int(num_tok / n)
    # get a more uniform splitting to avoid splits
    # which are too short at the end
    if len_chunk+50 > config.MODEL_MAX_LEN:
        len_chunk = int(num_tok / (n+1))
    
    len_curr = 0
    text_curr = []
    text_chunk = []
    for te, len_sent in zip(text_sent, num_tok_sent):

        if len_curr + len_sent < len_chunk:
            text_curr.append(te)
            len_curr += len_sent

        elif len_curr + len_sent >= config.MODEL_MAX_LEN:
            text_chunk.append(text_curr)

            text_curr = [te]
            len_curr = len_sent

        else: # >= len_chunk && < MODEL_MAX_LEN
            text_curr.append(te)
            text_chunk.append(text_curr)
            
            text_curr = []
            len_curr = 0

    if len_curr > 0:
        text_chunk.append(text_curr)

    return text_chunk

In [12]:
df['text_chunk'] = df.text.map(chunk_text)

flatten = lambda t: [item for sublist in t for item in sublist]
assert ''.join(flatten(df.text.tolist())) == ''.join(flatten(flatten(df.text_chunk.tolist())))

In [13]:
df.text = df.text.map(lambda t: ''.join(t))
df.text_chunk = df.text_chunk.map(lambda t: [''.join(c) for c in t])

In [14]:
df = df.drop(columns='text')
df = df.rename(columns={'text_chunk': 'text'})

#### Save new dataset

In [15]:
df.to_csv(OUTPUT_PATH+'df.csv')

#### Create train, test, validation

Generate files:
* train.source
* train.target
* val.source
* val.target
* test.source
* test.target

Since we do not have a lot of samples, validation and test will be equal for us.

In [16]:
df.bullets = df.bullets.map(lambda b: ' '.join(b))

In [17]:
df = df.sample(frac=1, random_state=config.SEED)
df['num_chunks'] = df.text.map(len).cumsum()
tot_chunk = df.num_chunks.iloc[-1]
split1 = np.where(df.num_chunks > int(tot_chunk*0.8))[0][0]+1
split2 = np.where(df.num_chunks > int(tot_chunk*0.9))[0][0]+1
print(split1, split2)

366 410


In [18]:
train, val, test =\
    df.iloc[:split1].explode('text'),\
    df.iloc[split1:split2].explode('text'),\
    df.iloc[split2:].explode('text')

train.to_csv(OUTPUT_PATH+'train.csv')
val.to_csv(OUTPUT_PATH+'val.csv')
test.to_csv(OUTPUT_PATH+'test.csv')

In [19]:
with open(OUTPUT_PATH+'train.source', 'w') as tr_s,\
    open(OUTPUT_PATH+'train.target', 'w') as tr_t,\
    open(OUTPUT_PATH+'train.index', 'w') as tr_i:
    for idx, row in train[['text', 'bullets']].iterrows():
        tr_i.write(str(idx) + '\n')
        tr_s.write(row.text + '\n')
        tr_t.write(row.bullets + '\n')

In [20]:
with open(OUTPUT_PATH+'val.source', 'w') as va_s,\
    open(OUTPUT_PATH+'val.target', 'w') as va_t,\
    open(OUTPUT_PATH+'val.index', 'w') as va_i:
    for idx, row in val[['text', 'bullets']].iterrows():
        va_i.write(str(idx) + '\n')
        va_s.write(row.text + '\n')
        va_t.write(row.bullets + '\n')

In [21]:
with open(OUTPUT_PATH+'test.source', 'w') as te_s,\
    open(OUTPUT_PATH+'test.target', 'w') as te_t,\
    open(OUTPUT_PATH+'test.index', 'w') as te_i:
    for idx, row in test[['text', 'bullets']].iterrows():
        te_i.write(str(idx) + '\n')
        te_s.write(row.text + '\n')
        te_t.write(row.bullets + '\n')